In [ ]:
from pyspark.sql.types import *
from pyspark.sql import Row
from time import time

# Introduction

Spark has the conceptual equivalent of R and pandas dataframes available through the DataFrames API. In Spark DataFrames may be created from existing RDDs, structured data files, external databases, or tables in Hive, etc. These DataFrames allow for the manipulation of distributed data through operations such as filter, groupby, and join, which are automatically optimized through parallelization and distribution across clusters when an action triggers the job.

We are going to use the dating profiles dataset for this tutorial. More information on this dataset can be found [here](https://sites.google.com/a/insightdatascience.com/spark-lab/s3-data/dating-profiles).

# Loading in the data
First, read in the data into an RDD. Here, we are using 16 partitions for our RDD.

In [ ]:
# Read in raw ratings data (fromUserId, toUserId, rating)
ratingsCsvRDD = sc.textFile("s3a://insight-spark-after-dark/ratings.csv.gz").repartition(12)

Reformat the data into a JSON format by applying the function rec_tup (which converts the data into a JSON format) to the RDD using the .map and .toDF() transformation steps.

In [ ]:
# Convert raw ratings RDD to Json RDD
def rec_tup(rating):
    tokens = rating.split(",")
    return Row(fromUserId=int(tokens[0]), toUserId=int(tokens[1]), rating=int(tokens[2]))

ratingsJson_DF = ratingsCsvRDD.map(rec_tup).toDF()
ratingsJson_DF.take(5)

As you might remember, Spark does a Lazy evaluation which means each transformed RDD may be recomputed each time you run an action on it. We'll use the .persist method (this is actually also a transformation) to keep the result (transformed RDD) on the cluster for quick future access.

**Note**: persist is a transformation, it will only run when a future action is called. One result of this is that if you run an action twice, you will persist twice (and right now in Spark you will lose the pointer to the first persist).

In [ ]:
# Cache the SchemaRDD as we'll be using this heavily moving forward
ratingsJson_DF.persist(StorageLevel.MEMORY_AND_DISK_SER)

Quickly check the schema of the dataframe. 
To write SparkSQL, we need to create a table object from our dataframe which we can use to run SparkSQL commands. The transformation registerTempTable() does this and we call our table 'ratingsJsonTable'.

In [ ]:
# Describe the SchemaRDD inferred from the JSON
ratingsJson_DF.printSchema()

# On to Spark DataFrames

Spark DataFrames is an abstract API that lets you perform Pandas or R like commands to your dataset.

In [ ]:
# Show the top 10 most-active users who are giving out ratings
mostActiveUsersSchemaDF = ratingsJson_DF.select("fromUserId","toUserId")\
                                        .groupBy("fromUserId")\
                                        .count()\
                                        .withColumnRenamed("count", "num_ratings")\
                                        .orderBy(["num_ratings"], ascending=[0])
mostActiveUsersSchemaDF.take(10)

# Sampling
We can also sample the dataframe with the sample() transformation. Great way to test the job before running on the full dataset.

Sample takes 3 arguments: **oversample (True/False), fraction of data in sample, seed**

In [ ]:
ratingsJson_DF.sample(False, 0.1, 20).collect()

# Next Steps

Here are some further questions to get to grips with Spark DataFrames.

### Task 1: Who are the top 5 users that gives the highest average rating of other profiles? 

### Task 2: How many pairs of users have rated each other with a rating > 5? (Is not intuitive...)